In [1]:
import pandas as pd
from  torch.utils.data import random_split,DataLoader
df = pd.read_csv('dataset.csv')
df['sentiment'] = df['sentiment'].map({'negative': 0, 'positive': 1})




In [2]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

class MovieDataset(Dataset):
    def __init__(self, dataframe, max_len=256):
        self.texts = dataframe['review'].tolist()
        self.labels = dataframe['sentiment'].tolist()
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = tokenizer.encode_plus(
            self.texts[idx],
            add_special_tokens=True,
            truncation=True,
            padding="max_length",   # automatically pads to max_len
            max_length=self.max_len,
            return_tensors="pt"
        )
        tokens = encoding['input_ids'].squeeze(0)  # remove batch dimension
        seq_len = (tokens != tokenizer.pad_token_id).sum().item()  # actual length
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return tokens, label, seq_len
dataset = MovieDataset(df)


In [3]:
from torch.utils.data import random_split

total_len = len(dataset)
train_len = int(0.8 * total_len)
valid_len = int(0.1 * total_len)
test_len  = total_len - train_len - valid_len

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_len, valid_len, test_len])

In [4]:
batch_size = 64
max_len = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)



In [5]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dim,
        hidden_dim,
        output_dim,
        n_layers,
        bidirectional,
        dropout_rate,
        pad_index,
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            bidirectional=bidirectional,
            dropout=dropout_rate if n_layers > 1 else 0.0,
            batch_first=True,
        )

        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.bidirectional = bidirectional

    def forward(self, ids, lengths):
        embedded = self.dropout(self.embedding(ids))
        
        # Pack the sequences
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        
        # Concatenate last hidden states from both directions if bidirectional
        if self.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2], hidden[-1]), dim=1))
        else:
            hidden = self.dropout(hidden[-1])
        
        prediction = self.fc(hidden)
        return prediction


In [6]:
import numpy as np
import tqdm
import torch

def train(data_loader, model, criterion, optimizer, device):
    model.train()
    epoch_losses = []
    epoch_accs = []

    for batch_ids, batch_labels, lengths in tqdm.tqdm(data_loader, desc="training..."):
        batch_ids = batch_ids.to(device)
        batch_labels = batch_labels.to(device)
        lengths = lengths.to(device)

        optimizer.zero_grad()
        prediction = model(batch_ids, lengths)
        loss = criterion(prediction, batch_labels)

        accuracy = (prediction.argmax(dim=1) == batch_labels).float().mean()

        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())
        epoch_accs.append(accuracy.item())

    return np.mean(epoch_losses), np.mean(epoch_accs)

def evaluate(data_loader, model, criterion, device):
    model.eval()
    epoch_losses = []
    epoch_accs = []

    with torch.no_grad():
        for batch_ids, batch_labels, lengths in tqdm.tqdm(data_loader, desc="evaluating..."):
            batch_ids = batch_ids.to(device)
            batch_labels = batch_labels.to(device)
            lengths = lengths.to(device)

            prediction = model(batch_ids, lengths)
            loss = criterion(prediction, batch_labels)
            accuracy = (prediction.argmax(dim=1) == batch_labels).float().mean()

            epoch_losses.append(loss.item())
            epoch_accs.append(accuracy.item())

    return np.mean(epoch_losses), np.mean(epoch_accs)


In [7]:
from torch.utils.data import DataLoader


# model
vocab_size = tokenizer.vocab_size
embedding_dim = 300
hidden_dim = 300
output_dim = 2
n_layers = 2
bidirectional = True
dropout_rate = 0.5
pad_index = tokenizer.pad_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LSTMModel = LSTM(
    vocab_size,
    embedding_dim,
    hidden_dim,
    output_dim,
    n_layers,
    bidirectional,
    dropout_rate,
    pad_index
).to(device)

# optimizer and loss
optimizer = torch.optim.Adam(LSTMModel.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
n_epochs = 10
best_valid_loss = float("inf")

for epoch in range(n_epochs):
    train_loss, train_acc = train(train_loader, LSTMModel, loss_fn, optimizer, device)
    valid_loss, valid_acc = evaluate(valid_loader, LSTMModel, loss_fn, device)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(LSTMModel.state_dict(), "best_lstm.pt")
    
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f} | Valid Loss: {valid_loss:.3f}, Valid Acc: {valid_acc:.3f}")


evaluating...: 100%|██████████| 79/79 [00:06<00:00, 11.40it/s]


Epoch 1 | Train Loss: 0.664, Train Acc: 0.597 | Valid Loss: 0.590, Valid Acc: 0.684


evaluating...: 100%|██████████| 79/79 [00:08<00:00,  9.35it/s]


Epoch 2 | Train Loss: 0.579, Train Acc: 0.696 | Valid Loss: 0.446, Valid Acc: 0.797


evaluating...: 100%|██████████| 79/79 [00:07<00:00, 11.11it/s]


Epoch 3 | Train Loss: 0.377, Train Acc: 0.834 | Valid Loss: 0.300, Valid Acc: 0.877


evaluating...: 100%|██████████| 79/79 [00:07<00:00, 10.90it/s]


Epoch 4 | Train Loss: 0.299, Train Acc: 0.875 | Valid Loss: 0.275, Valid Acc: 0.884


training...:  14%|█▍        | 88/625 [00:15<01:32,  5.84it/s]